In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
# added later for linregress
from scipy.stats import linregress

In [2]:
fileone = "../Resources/police_reports_clean.csv"
filetwo = "../Resources/zillow_regression.csv"

police_reports = pd.read_csv(fileone)
zillow = pd.read_csv(filetwo)

In [12]:
def extract_year_month(date_str):
    return "-".join(date_str.split("-")[:2])
police_reports["Incident Year-Month"] = police_reports["Incident Date"].apply(extract_year_month)
police_reports["Report Year-Month"] = police_reports["Incident Date"].apply(extract_year_month)
police_reports.head()

,Incident Datetime,Incident Date,Incident Year,Incident Day of Week,Report Datetime,Report Type Description,Filed Online,Incident Category,Incident Subcategory,Incident Description,...,Intersection,Police District,Analysis Neighborhood,Latitude,Longitude,Incident Time,Report Time,Report Date,Incident Year-Month,Report Year-Month
0,2021-08-03 10:00:00,2021-08-03,2021,Tuesday,2021-08-15 15:40:00,Initial,False,Fraud,Fraud,Fraudulent Use Of Automated Teller Card,...,04TH ST \ HOWARD ST,Southern,Financial District/South Beach,37.783259,-122.402708,10:00:00,15:40:00,2021-08-15,2021-08,2021-08
1,2021-11-23 07:38:00,2021-11-23,2021,Tuesday,2021-11-23 11:14:00,Initial,False,Arson,Arson,Arson,...,FILLMORE ST \ PAGE ST,Northern,Hayes Valley,37.773039,-122.430629,07:38:00,11:14:00,2021-11-23,2021-11,2021-11
2,2018-03-16 19:45:00,2018-03-16,2018,Friday,2018-03-16 23:30:00,Initial,False,Assault,Simple Assault,Battery,...,JONES ST \ POST ST,Central,Tenderloin,37.787664,-122.413354,19:45:00,23:30:00,2018-03-16,2018-03,2018-03
3,2019-09-03 08:25:00,2019-09-03,2019,Tuesday,2019-09-04 14:53:00,Initial,False,Lost Property,Lost Property,Lost Property,...,07TH ST \ BRANNAN ST,Southern,South of Market,37.773051,-122.403205,08:25:00,14:53:00,2019-09-04,2019-09,2019-09
4,2020-01-01 00:00:00,2020-01-01,2020,Wednesday,2021-06-23 13:49:00,Initial,False,Fraud,Fraud,"False Claims, Presenting to Government",...,47TH AVE \ WAWONA ST,Taraval,Sunset/Parkside,37.736135,-122.505297,00:00:00,13:49:00,2021-06-23,2020-01,2020-01


In [30]:
violent_crimes = ["Assault", "Larceny Theft", "Burglary", "Motor Vehicle Theft", "Robbery",
                  "Weapons Offense", "Arson", "Rape", "Homicide",
                  "Human Trafficking"]

nonviolent_crimes = ["Malicious Mischief", "Other Miscellaneous", "Fraud", "Drug Offense",
                     "Disorderly Conduct", "Offences Against The Family And Children",
                     "Traffic Violation Arrest", "Weapons Carrying Etc",
                     "Stolen Property", "Forgery and Counterfeiting", "Traffic Collision",
                     "Vandalism", "Prostitution", "Embezzlement", "Civil Sidewalks",
                     "Sex Offense", "Drug Violation", "Gambling"]

violent_crimes_df = police_reports.loc[police_reports["Incident Category"].isin(violent_crimes)]
nonviolent_crimes_df = police_reports.loc[police_reports["Incident Category"].isin(nonviolent_crimes)]

In [31]:
violent_crimes_df = pd.DataFrame(violent_crimes_df)
nonviolent_crimes_df = pd.DataFrame(nonviolent_crimes_df)
violent_crimes_df

,Incident Datetime,Incident Date,Incident Year,Incident Day of Week,Report Datetime,Report Type Description,Filed Online,Incident Category,Incident Subcategory,Incident Description,...,Intersection,Police District,Analysis Neighborhood,Latitude,Longitude,Incident Time,Report Time,Report Date,Incident Year-Month,Report Year-Month
1,2021-11-23 07:38:00,2021-11-23,2021,Tuesday,2021-11-23 11:14:00,Initial,False,Arson,Arson,Arson,...,FILLMORE ST \ PAGE ST,Northern,Hayes Valley,37.773039,-122.430629,07:38:00,11:14:00,2021-11-23,2021-11,2021-11
2,2018-03-16 19:45:00,2018-03-16,2018,Friday,2018-03-16 23:30:00,Initial,False,Assault,Simple Assault,Battery,...,JONES ST \ POST ST,Central,Tenderloin,37.787664,-122.413354,19:45:00,23:30:00,2018-03-16,2018-03,2018-03
7,2021-06-13 13:00:00,2021-06-13,2021,Sunday,2021-06-13 13:13:00,Initial,False,Assault,Simple Assault,Battery,...,MISSION ST \ 25TH ST,Mission,Mission,37.750640,-122.418288,13:00:00,13:13:00,2021-06-13,2021-06,2021-06
9,2021-10-11 20:45:00,2021-10-11,2021,Monday,2021-10-11 20:54:00,Initial,False,Assault,Simple Assault,Battery,...,TAYLOR ST \ EDDY ST,Tenderloin,Tenderloin,37.784141,-122.410952,20:45:00,20:54:00,2021-10-11,2021-10,2021-10
10,2021-10-12 05:40:00,2021-10-12,2021,Tuesday,2021-10-12 06:20:00,Initial,False,Larceny Theft,Larceny - Auto Parts,"Theft, Vehicle Strip, >$950",...,VALLEJO ST \ JONES ST,Central,Nob Hill,37.797750,-122.415384,05:40:00,06:20:00,2021-10-12,2021-10,2021-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411289,2021-05-17 14:30:00,2021-05-17,2021,Monday,2021-05-18 22:40:00,Vehicle Initial,False,Motor Vehicle Theft,Motor Vehicle Theft,"Vehicle, Stolen, Other Vehicle",...,ARGUELLO BLVD \ CLEMENT ST,Richmond,Inner Richmond,37.783189,-122.458998,14:30:00,22:40:00,2021-05-18,2021-05,2021-05
411290,2021-10-07 12:00:00,2021-10-07,2021,Thursday,2021-10-07 14:25:00,Initial,False,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, Att.",...,WEBSTER ST \ POST ST,Northern,Japantown,37.785373,-122.431366,12:00:00,14:25:00,2021-10-07,2021-10,2021-10
411291,2021-11-21 11:11:00,2021-11-21,2021,Sunday,2021-11-22 23:21:00,Initial,False,Burglary,Burglary - Hot Prowl,"Burglary, Hot Prowl, Unlawful Entry",...,LAGUNA ST \ ROSE ST,Northern,Hayes Valley,37.773200,-122.425606,11:11:00,23:21:00,2021-11-22,2021-11,2021-11
411293,2021-05-19 15:00:00,2021-05-19,2021,Wednesday,2021-05-20 07:09:00,Vehicle Initial,False,Motor Vehicle Theft,Motor Vehicle Theft,"Vehicle, Stolen & Recovered",...,SANTIAGO ST \ 24TH AVE,Taraval,Sunset/Parkside,37.744681,-122.481181,15:00:00,07:09:00,2021-05-20,2021-05,2021-05


In [32]:
violent_groupby = violent_crimes_df.groupby(["Analysis Neighborhood","Incident Year-Month"]).count()["Incident Datetime"]
nonviolent_groupby = nonviolent_crimes_df.groupby(["Analysis Neighborhood","Incident Year-Month"]).count()["Incident Datetime"]

In [33]:
violent_grouped = pd.DataFrame(violent_groupby)
nonviolent_grouped = pd.DataFrame(nonviolent_groupby)

In [34]:
violent_df = violent_grouped.rename(columns = {"Analysis Neighborhood":"Neighborhood","Incident Year-Month":"Date","Incident Category": "Number of Crimes"})
nonviolent_df = nonviolent_grouped.rename(columns = {"Analysis Neighborhood":"Neighborhood","Incident Year-Month":"Date","Incident Category": "Number of Crimes"})

In [35]:
violent_df = pd.DataFrame(violent_df)
nonviolent_df = pd.DataFrame(nonviolent_df)

In [36]:
violent_df = violent_df.reset_index()
nonviolent_df = nonviolent_df.reset_index()
nonviolent_df

,Analysis Neighborhood,Incident Year-Month,Incident Datetime
0,Bayview Hunters Point,2018-01,153
1,Bayview Hunters Point,2018-02,103
2,Bayview Hunters Point,2018-03,144
3,Bayview Hunters Point,2018-04,134
4,Bayview Hunters Point,2018-05,118
...,...,...,...
2407,Western Addition,2022-08,64
2408,Western Addition,2022-09,59
2409,Western Addition,2022-10,65
2410,Western Addition,2022-11,52


In [37]:
violent_df.to_csv("../Resources/violent_crimes_regression.csv", index=False, header=True)
nonviolent_df.to_csv("../Resources/nonviolent_crimes_regression.csv", index=False, header=True)

In [ ]:
# making csv for regression by crime category